In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36")
}

resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

# Look through each <table>, try to parse it and pick the one that looks like revenue data.
tesla_revenue = None
for table in soup.find_all("table"):
    try:
        df = pd.read_html(str(table))[0]
    except ValueError:
        continue  # not a parseable table
    cols = [str(c).lower() for c in df.columns.astype(str)]
    # heuristic: table has 'revenue' in a column name and has at least two columns (date/quarter + value)
    if any("revenue" in c for c in cols) and len(cols) >= 2:
        tesla_revenue = df.copy()
        break

# Fallback: try reading all tables from the page and pick one that contains "Revenue" in column names
if tesla_revenue is None:
    dfs = pd.read_html(resp.text)
    for df in dfs:
        cols = [str(c).lower() for c in df.columns.astype(str)]
        if any("revenue" in c for c in cols) and len(cols) >= 2:
            tesla_revenue = df.copy()
            break

if tesla_revenue is None:
    raise RuntimeError("Could not find Tesla revenue table on the page. The page structure may have changed.")

# Keep only the first two columns (common layout: Quarter/Date and Revenue) and normalize names
tesla_revenue = tesla_revenue.iloc[:, :2]
tesla_revenue.columns = ["Quarter", "Revenue"]

# Clean revenue numbers (remove $ and commas) and drop empty rows
tesla_revenue["Revenue"] = (
    tesla_revenue["Revenue"]
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)
    .str.strip()
)
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""].dropna().reset_index(drop=True)

# Ensure numeric type for Revenue (optional)
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce")

# Show the last five rows
print(tesla_revenue.tail())
